In [1]:
import torch
import numpy as np
import time
import math
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import pandas as pd

In [2]:
device = torch.device("cuda") 
def formatLabels(tens, num_outputs):
    output = torch.zeros(tens.size()[0], num_outputs)
    output = output.float()
    output = output.to(device)
    
    for val, i in enumerate(tens):
        output[val][int(i)] = 1.0
    return output.long()

test = torch.tensor([7., 6., 6., 5., 6., 6., 2., 1.])
formatLabels(test, 10)


tensor([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')

In [3]:
device = torch.device("cuda") 
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', delimiter=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)
train_dataset = data.TensorDataset(torch.from_numpy(train.values[:,:-1]),torch.from_numpy(train.values[:,-1]))
train_data_loader = data.DataLoader(train_dataset,batch_size=8, shuffle=True)
test_dataset = data.TensorDataset(torch.from_numpy(test.values[:,:-1]),torch.from_numpy(test.values[:,-1]))
test_data_loader = data.DataLoader(test_dataset,batch_size=8, shuffle=False)

In [5]:
def EqualSampleByLabel(tens):
    tens = tens[tens[:,-1].argsort()]
    tens = np.split(tens, list(np.unique(tens.astype(int)[:, -1], return_index=True)[1][1:]))
    return tens

t = train.to_numpy()
#EqualSampleByLabel(t)

In [6]:
def CreateSyntheticData(tensor, size, stddev = 1, scale = 0.05, lastIsKeys = True):
    gauss = torch.normal(torch.zeros(size))
    tensor[-1] = 0.0
    corrected = tensor*scale*gauss + tensor
    return corrected

a = torch.ones(5)
CreateSyntheticData(a, 5)

def GaussianSyntheticNeighbours(dataset, objetiveLen):
    return dataset.size(dim=0)

a = torch.ones(5,4)
GaussianSyntheticNeighbours(a, 20)

5

In [7]:
class WineClassifier(nn.Module):

    def __init__(self, num_inputs= 11, num_hidden = 15, num_outputs=10):
        super().__init__()
        # Initialize the modules we need to build the network
        self.linear1 = nn.Linear(num_inputs, num_hidden)
        self.act_fn1 = nn.Sigmoid()
        self.linear2 = nn.Linear(num_hidden, num_hidden)
        self.act_fn2 = nn.Sigmoid()
        self.linear3 = nn.Linear(num_hidden, num_outputs)                

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.linear1(x)
        x = self.act_fn1(x)
        x = self.linear2(x)
        x = self.act_fn2(x)
        x = self.linear3(x)

        return x

In [8]:
params = (11,15,1) # num_inputs, num_hidden, num_outputs
model = WineClassifier(*params)
model = model.to(device)

In [9]:
#optimizer = torch.optim.SGD(model.parameters(), lr = 0.000001, momentum=0.2)
optimizer = torch.optim.Adam(model.parameters(), lr =0.0004)

# criterion = torch.nn.CrossEntropyLoss()
# criterion = nn.MSELoss(reduction = "sum")
criterion = nn.MSELoss()
# criterion = F.l1_loss()
# criterion = nn.L1Loss()

model.train() 

# Training loop
for epoch in range(1500):
    optimizer = torch.optim.Adam(model.parameters(), lr =0.0004/(epoch+1))
    for data_inputs, data_labels in train_data_loader:

        data_inputs, data_labels = data_inputs.float().to(device), data_labels.float().to(device)
        preds = model(data_inputs).to(device)
        preds = preds.squeeze(dim=1)

        loss = criterion(preds, data_labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}")

Epoch: 0, loss: 17.4
Epoch: 100, loss: 0.879
Epoch: 200, loss: 0.622
Epoch: 300, loss: 0.59
Epoch: 400, loss: 0.684
Epoch: 500, loss: 0.516
Epoch: 600, loss: 0.588
Epoch: 700, loss: 0.914
Epoch: 800, loss: 1.48
Epoch: 900, loss: 0.622
Epoch: 1000, loss: 1.2
Epoch: 1100, loss: 0.608
Epoch: 1200, loss: 0.203
Epoch: 1300, loss: 0.188
Epoch: 1400, loss: 0.23


In [10]:
model.eval() # Set model to eval mode
true_preds, num_preds = 0., 0.

with torch.no_grad(): # Deactivate gradients for the following code
    for data_inputs, data_labels in test_data_loader:

        # Determine prediction of model on dev set
        data_inputs = data_inputs.float()
        data_inputs, data_labels = data_inputs.to(device), data_labels.to(device)
        preds = model(data_inputs)
        preds = preds.squeeze(dim=1)
        print(torch.round(preds)); print(data_labels)
        pred_labels = torch.round(preds)

        # Keep records of predictions for the accuracy metric (true_preds=TP+TN, num_preds=TP+TN+FP+FN)
        true_preds += (pred_labels == data_labels).sum()
        num_preds += data_labels.shape[0]

acc = true_preds / num_preds
print(f"Accuracy of the model: {100.0*acc:4.2f}%")

tensor([6., 6., 6., 6., 6., 6., 6., 6.], device='cuda:0')
tensor([6., 8., 6., 5., 6., 5., 6., 6.], device='cuda:0', dtype=torch.float64)
tensor([6., 6., 6., 6., 6., 6., 6., 6.], device='cuda:0')
tensor([6., 6., 5., 5., 7., 7., 6., 5.], device='cuda:0', dtype=torch.float64)
tensor([6., 6., 6., 6., 6., 6., 6., 6.], device='cuda:0')
tensor([5., 5., 5., 6., 5., 6., 5., 5.], device='cuda:0', dtype=torch.float64)
tensor([6., 6., 6., 6., 6., 6., 6., 6.], device='cuda:0')
tensor([6., 6., 6., 5., 6., 8., 5., 6.], device='cuda:0', dtype=torch.float64)
tensor([6., 6., 6., 6., 6., 6., 6., 6.], device='cuda:0')
tensor([7., 5., 6., 4., 4., 5., 5., 8.], device='cuda:0', dtype=torch.float64)
tensor([6., 6., 6., 6., 6., 6., 6., 6.], device='cuda:0')
tensor([6., 5., 6., 5., 7., 5., 6., 4.], device='cuda:0', dtype=torch.float64)
tensor([6., 6., 6., 6., 6., 6., 6., 6.], device='cuda:0')
tensor([7., 5., 5., 6., 6., 6., 5., 6.], device='cuda:0', dtype=torch.float64)
tensor([6., 6., 6., 6., 6., 6., 6., 6.], 